# **Generate all figures**

In [1]:
import pandas as pd
import pyproj
from ReportTemplates.MapsPlotter import MapsPlotter

# allBuildingsListPath = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Data\Building_identification_data_Just data.csv"
allBuildingsListPath = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\_Final report\Building_identification_data_EAZK_v6_CZE_English.csv"
resultsPath = "C:/Users/jaasb/INVESTIGO/BEE Group/eplanet shared/Programa Final/_Final Results/"
mapResultsPath = "C:/Users/jaasb/INVESTIGO/BEE Group/eplanet shared/Programa Final/Samples and Test/Automated Report Test/MapResults/"
allBuildings = pd.read_csv(allBuildingsListPath, delimiter=";", encoding='UTF-8')


def signFromDirection(value):
    direction = value[-1]
    number = value[:-1]
    if direction == 'E' or direction == 'N':    return float(number)
    elif direction == 'W' or direction == 'S':  return -float(number)
    else:   return value
    
source=4326
target=5514
# Reproject the remaining buildings
source_crs = 'epsg:' + str(source) # Coordinate system of the buildings (default=global)
target_crs = 'epsg:' + str(target) # Coordinate system of LiDAR data (default=Czech)

global_to_czech = pyproj.Transformer.from_crs(source_crs, target_crs)
allBuildings['x'] = global_to_czech.transform(allBuildings.lat.apply(signFromDirection), allBuildings.long.apply(signFromDirection))[0]
allBuildings['y'] = global_to_czech.transform(allBuildings.lat.apply(signFromDirection), allBuildings.long.apply(signFromDirection))[1]

revisionBuildings = ['eP-EAZK-084']
allBuildings = allBuildings[allBuildings['identifier'].isin(revisionBuildings)].reset_index(drop=True)
allBuildings

,identifier,Country,Region,Municipality,Road,Road number,Postal code,long,lat,Name,...,Zdroj vytápění,Původní instalovaný výkon,Nominal lighting power (kW),DHW source,Instalovaný výkon po opatřeních,Cooling source,Power (kW),Unnamed: 35,x,y
0,eP-EAZK-084,Czech Republic,Zlínský,Nový Hrozenkov,-,437,756 04,18.1987378E,49.3388056N,Základní škola - budova B,...,kondenzační plynový kotel,906 kW,NaN,NaN,"21,2 kW",NaN,NaN,NaN,-481396.082237,-1.156083e+06


In [4]:
from PIL import Image
import os

def optimize_Overwrite(image, destination, size):
    if(os.path.isfile(image)):
        foo = Image.open(image)
        foo = foo.resize((size, size))
        foo.save(destination, optimize=True, quality = 10)

images = ["_ZoomOut.png", "_ZoomIn.png", "_Planes.png", "_LiDAR.png", "_EnergyDensity.png", "_EnergyNumbered.png"]
size = [333, 200, 500, 500, 500, 500]

for i in range(len(allBuildings)):
# for i in range(1):
    building = allBuildings.iloc[[i]].reset_index(drop=True)
    print(building.identifier.values[0])
    plotter = MapsPlotter(resultsPath, mapResultsPath)
    plotter.zoomOutMap(building)
    plotter.zoomInMap(building)
    plotter.LiDARscatter(building)
    plotter.plotPlanes(building)
    plotter.plotEnergy(building, numbered=False, offsetX=0, offsetY=0)
    plotter.plotEnergy(building, numbered=True, offsetX=0, offsetY=0)
    for i in range(len(images)):
        image = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Samples and Test\Automated Report Test\MapResults" + "/" + building.identifier.values[0] + images[i]
        destination = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Samples and Test\Automated Report Test\MapResults_Compressed" + "/" + building.identifier.values[0] + images[i]
        optimize_Overwrite(image, destination, size[i])

eP-EAZK-084


# **Global variables**

In [8]:
reportTitle = "ePlanet - UC4: PV potential analysis for public buildings, 25/03/2024"
logo = "eplanet-fullcolor.png"

# **Apply format to introduction pages**

In [15]:
from ReportTemplates.CanvasWriter import CanvasWriter
# Contents of this report
originPath = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\_Final report\Contents of this report.pdf"
destinationPath = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\_Final report\Contents of this report - Watermarked.pdf"
startingPage = 11
CanvasWriter.watermark(originPath, destinationPath, reportTitle, logo, startingPage)
# Buildings summary
originPath = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\_Final report\Buildings summary.pdf"
destinationPath = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\_Final report\Buildings summary - Watermarked.pdf"
startingPage = 13
CanvasWriter.watermark(originPath, destinationPath, reportTitle, logo, startingPage)

# ***(deprecated)* Generate summary table from csv**

In [4]:
# import pandas as pd
# summaryTabledf = pd.read_csv(r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\_Final report\Building_identification_data_EAZK_v6_CZE_English.csv", sep=";", encoding="UTF-8")

# # for i in range(len(summaryTabledf)):
# #     if(type(summaryTabledf[["Unnamed: 9"]].iloc[i].values[0]) == type("sample text")):
# #        summaryTabledf.iloc[i, 8] = summaryTabledf[["Simulation state"]].iloc[i].values[0] + ' ' + summaryTabledf[["Unnamed: 9"]].iloc[i].values[0]

# # summaryTabledf = summaryTabledf.drop(columns=["Unnamed: 9"])
# maxchars = 60
# summaryTabledf = summaryTabledf[["identifier", "Municipality", "Road", "Road number", "long", "lat", "Name", "Simulation results"]]
# for i in range(len(summaryTabledf)):
#     # summaryTabledf.loc[["Name"], i] = 
#     name = summaryTabledf.Name[i].split(" - ")
#     if(len(name) > 1):
#         summaryTabledf.Name[i] = ' - '.join(name[0:len(name)-1])

#     if(len(summaryTabledf.Name[i]) > maxchars):
#         summaryTabledf.Name[i] = summaryTabledf.Name[i][0:maxchars] + "..."
# summaryTabledf

In [5]:
# from ReportTemplates.ReportTypes import ReportTypes

# pagenumber = 13

# allBuildingsListPath = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\_Final report\Building_identification_data_EAZK_v6_CZE_English.csv"
# resultsPath = "C:/Users/jaasb/INVESTIGO/BEE Group/eplanet shared/Programa Final/_Final Results/"
# mapResultsPath = "C:/Users/jaasb/INVESTIGO/BEE Group/eplanet shared/Programa Final/Samples and Test/Automated Report Test/MapResults_Compressed/"
# reporter = ReportTypes(reportTitle, logo, pagenumber, resultsPath, mapResultsPath)

# destinationPath = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\_Final report\tablePages.pdf"

# reporter.summaryReport(summaryTabledf, destinationPath)

# **Simultation report generation**
For all buildings.
And merged

In [14]:
import pandas as pd
import pyproj

allBuildingsListPath = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\_Final report\Building_identification_data_EAZK_v6_CZE_English.csv"
resultsPath = "C:/Users/jaasb/INVESTIGO/BEE Group/eplanet shared/Programa Final/_Final Results/"
mapResultsPath = "C:/Users/jaasb/INVESTIGO/BEE Group/eplanet shared/Programa Final/Samples and Test/Automated Report Test/MapResults/"

allBuildings = pd.read_csv(allBuildingsListPath, delimiter=";", encoding='UTF-8')

def signFromDirection(value):
    direction = value[-1]
    number = value[:-1]
    if direction == 'E' or direction == 'N':    return float(number)
    elif direction == 'W' or direction == 'S':  return -float(number)
    else:   return value
    
source=4326
target=5514
# Reproject the remaining buildings
source_crs = 'epsg:' + str(source) # Coordinate system of the buildings (default=global)
target_crs = 'epsg:' + str(target) # Coordinate system of LiDAR data (default=Czech)

global_to_czech = pyproj.Transformer.from_crs(source_crs, target_crs)
allBuildings['x'] = global_to_czech.transform(allBuildings.lat.apply(signFromDirection), allBuildings.long.apply(signFromDirection))[0]
allBuildings['y'] = global_to_czech.transform(allBuildings.lat.apply(signFromDirection), allBuildings.long.apply(signFromDirection))[1]

# allBuildings = allBuildings[allBuildings['identifier'].isin(['eP-EAZK-332', 'eP-EAZK-333'])].reset_index(drop=True)
allBuildings

,identifier,Country,Region,Municipality,Road,Road number,Postal code,long,lat,Name,...,Zdroj vytápění,Původní instalovaný výkon,Nominal lighting power (kW),DHW source,Instalovaný výkon po opatřeních,Cooling source,Power (kW),Unnamed: 35,x,y
0,eP-EAZK-001,Czech Republic,Zlínský,Hluk,Sokolská,655,687 25,17.5292961E,48.9891172N,Kino Hluk,...,kondenzační plynový kotel,2 x 33 kW,NaN,NaN,2 x 32 kW,NaN,NaN,NaN,-533550.698635,-1.190366e+06
1,eP-EAZK-002,Czech Republic,Zlínský,Karolinka,Kobylská,391,756 05,18.2310311E,49.3541422N,Školní budova,...,kondenzační plynový kotel,-,NaN,NaN,24 kW,NaN,NaN,NaN,-478911.148623,-1.154587e+06
2,eP-EAZK-003,Czech Republic,Zlínský,Karolinka,Nábřežní,175,756 05,18.2401997E,49.3526650N,Objekt pro charitu,...,kondenzační plynový kotel,2 x 43 kW,NaN,NaN,-,NaN,NaN,NaN,-478261.692215,-1.154808e+06
3,eP-EAZK-004,Czech Republic,Zlínský,Bánov,-,428,687 54,17.7181519E,48.9880614N,Hasičská zbrojnice,...,kondenzační plynový kotel,"25,23 a 9,9 kW",NaN,NaN,-,NaN,NaN,NaN,-519802.405316,-1.191784e+06
4,eP-EAZK-005,Czech Republic,Zlínský,Pozlovice,Hlavní,361,763 26,17.7634753E,49.1210800N,Šatny pro sportovce,...,plynová topidla a elektrické infrazářiče,"71,4 kW",NaN,NaN,-,NaN,NaN,NaN,-515134.282103,-1.177363e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,eP-EAZK-373,Czech Republic,Zlínský,Valašské Meziříčí,Sokolská,-,757 01,17.9652806E,49.4706822N,Muzeum regionu Valašsko - kostel sv. Trojice,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-496980.157178,-1.139981e+06
373,eP-EAZK-374,Czech Republic,Zlínský,Valašské Meziříčí,Zámecká,3/55,757 01,17.9683628E,49.4784300N,Muzeum regionu Valašsko - zámek Kinských - hla...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-496680.453427,-1.139143e+06
374,eP-EAZK-375,Czech Republic,Zlínský,Valašské Meziříčí,Zámecká,3/55,757 01,17.9683628E,49.4784300N,Muzeum regionu Valašsko - zámek Kinských - pří...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-496680.453427,-1.139143e+06
375,eP-EAZK-376,Czech Republic,Zlínský,Vsetín,Horní náměstí,2,755 01,17.9994386E,49.3406953N,Muzeum regionu Valašsko - zámek Vsetín,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-495804.108123,-1.154601e+06


In [15]:
import os
import shutil

def create_output_folder(directory, deleteFolder=False):
    """
    Creates a folder for the given directory, only if it didn't exist before.
    If the directory already existed, it does nothing
    """

    if not(os.path.isdir(directory)):
       os.makedirs(directory)
    else:
        if(deleteFolder):
            shutil.rmtree(directory) # Deletes directory and its contents
            os.makedirs(directory)  # Creates new directory    

def find_index_title(buildingID, repeat_groups):
    for i in range(len(repeat_groups)):
        for j in range(len(repeat_groups[i])):
            if(buildingID == repeat_groups[i][j]):
                # It's in, now we need to find which is the one with data
                pathToCheck = resultsPath + buildingID
                if(os.path.isdir(pathToCheck)):
                    return ', '.join(repeat_groups[i])
                else:
                    return False

    return buildingID

impossiblebuildingsPath = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Results\Data Preparation\Buildings\impossible_buildings.txt"
impossiblebuildings = pd.read_csv(impossiblebuildingsPath)

equalbuildingsPath = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Results\Data Preparation\Buildings\equal_buildings.txt"
equalbuildings = pd.read_csv(equalbuildingsPath)
result = equalbuildings.groupby('equal to')['identifier'].apply(lambda x: ', '.join(x)).reset_index()

repeat_groups = []
for index, row in result.iterrows():
    input_string = row['equal to'] + ', ' + row['identifier']
    elements = input_string.split(', ')
    sorted_elements = sorted(elements)
    repeat_groups.append(sorted_elements)

splitBuildingsPath = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Results\Data Preparation\Buildings\SplitBuildings.txt"
splitBuildings = pd.read_csv(splitBuildingsPath)

for buildingSplit in splitBuildings["from"].unique():
    subdf = splitBuildings[splitBuildings["from"] == buildingSplit].reset_index(drop=True)
    for i in range(len(subdf)):
        newRow = allBuildings[allBuildings.identifier == buildingSplit]
        newRow.lat = subdf.lat[i]
        newRow.long = subdf.long[i]
        newRow.identifier = subdf.new[i]
        allBuildings = pd.concat([allBuildings, newRow])
    allBuildings = allBuildings.sort_values("identifier").reset_index(drop=True)
    allBuildings = allBuildings.drop(allBuildings[allBuildings["identifier"] == buildingSplit].index)

allBuildings = allBuildings.sort_values("identifier").reset_index(drop=True)
allBuildings

,identifier,Country,Region,Municipality,Road,Road number,Postal code,long,lat,Name,...,Zdroj vytápění,Původní instalovaný výkon,Nominal lighting power (kW),DHW source,Instalovaný výkon po opatřeních,Cooling source,Power (kW),Unnamed: 35,x,y
0,eP-EAZK-001,Czech Republic,Zlínský,Hluk,Sokolská,655,687 25,17.5292961E,48.9891172N,Kino Hluk,...,kondenzační plynový kotel,2 x 33 kW,NaN,NaN,2 x 32 kW,NaN,NaN,NaN,-533550.698635,-1.190366e+06
1,eP-EAZK-002,Czech Republic,Zlínský,Karolinka,Kobylská,391,756 05,18.2310311E,49.3541422N,Školní budova,...,kondenzační plynový kotel,-,NaN,NaN,24 kW,NaN,NaN,NaN,-478911.148623,-1.154587e+06
2,eP-EAZK-003,Czech Republic,Zlínský,Karolinka,Nábřežní,175,756 05,18.2401997E,49.3526650N,Objekt pro charitu,...,kondenzační plynový kotel,2 x 43 kW,NaN,NaN,-,NaN,NaN,NaN,-478261.692215,-1.154808e+06
3,eP-EAZK-004,Czech Republic,Zlínský,Bánov,-,428,687 54,17.7181519E,48.9880614N,Hasičská zbrojnice,...,kondenzační plynový kotel,"25,23 a 9,9 kW",NaN,NaN,-,NaN,NaN,NaN,-519802.405316,-1.191784e+06
4,eP-EAZK-005,Czech Republic,Zlínský,Pozlovice,Hlavní,361,763 26,17.7634753E,49.1210800N,Šatny pro sportovce,...,plynová topidla a elektrické infrazářiče,"71,4 kW",NaN,NaN,-,NaN,NaN,NaN,-515134.282103,-1.177363e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,eP-EAZK-373,Czech Republic,Zlínský,Valašské Meziříčí,Sokolská,-,757 01,17.9652806E,49.4706822N,Muzeum regionu Valašsko - kostel sv. Trojice,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-496980.157178,-1.139981e+06
381,eP-EAZK-374,Czech Republic,Zlínský,Valašské Meziříčí,Zámecká,3/55,757 01,17.9683628E,49.4784300N,Muzeum regionu Valašsko - zámek Kinských - hla...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-496680.453427,-1.139143e+06
382,eP-EAZK-375,Czech Republic,Zlínský,Valašské Meziříčí,Zámecká,3/55,757 01,17.9683628E,49.4784300N,Muzeum regionu Valašsko - zámek Kinských - pří...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-496680.453427,-1.139143e+06
383,eP-EAZK-376,Czech Republic,Zlínský,Vsetín,Horní náměstí,2,755 01,17.9994386E,49.3406953N,Muzeum regionu Valašsko - zámek Vsetín,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-495804.108123,-1.154601e+06


In [17]:
from ReportTemplates.DataManager import DataManager
from ReportTemplates.ReportTypes import ReportTypes

eachPDF = []

create_output_folder("PDF docs/Construction")
pd.set_option('mode.chained_assignment', None)

pagenumber = 29

reporter = ReportTypes(reportTitle, logo, pagenumber, resultsPath, mapResultsPath, numberPages=False)

for i in range(len(allBuildings)): 
    
    building = allBuildings.iloc[[i]].reset_index(drop=True)
    buildingName = building.identifier.values[0]

    print(buildingName)

    title = find_index_title(buildingName, repeat_groups)
   
    if(title != False):
        for j in range(len(repeat_groups)):
            if(buildingName in repeat_groups[j]):
                names = allBuildings[allBuildings.identifier.isin(repeat_groups[j])].Name.values
                building.Name[0] = building.Name[0].split(" - ")[0]
        
        planeListPath = resultsPath + buildingName + "/04 - Plane Processing/01 - Results/PlaneList_" + buildingName + ".csv"
        if(os.path.isfile(planeListPath)):
            planedf = pd.read_csv(planeListPath)

            planePointList = []
            planedf["z"] = 0.0
            for j in range(len(planedf)):
                currentFile = resultsPath + buildingName + "/04 - Plane Processing/01 - Results/" + buildingName + "_"+ str(j) + ".csv"
                df = pd.read_csv(currentFile, header=None)
                df = df.rename(columns={0:"x", 1:"y", 2:"z"})
                planePointList.append(df)
                planedf["z"].iloc[j] = df.z.mean()
            fields = ["area", "z", "tilt", "azimuth"]

        sunSimulationPath = resultsPath + buildingName + "/06 - PySAM Simulation/Summary.csv"
        reporter.updateData(planeListPath, sunSimulationPath, planedf, fields, planePointList)

        if(os.path.isfile(sunSimulationPath)):
            sundf = DataManager.prepare_sundf(sunSimulationPath)
            summaryEnergydf = DataManager.prepare_summaryEnergydf(sundf)
            sunByRoofList = DataManager.prepare_sunByRoofList(sundf)
            
            if(len(planedf) > 12):
                reporter.large_file(title, building)
            else:
                reporter.normal_file(title, building)
            
            eachPDF.append("PDF docs - Not Paged/" + title + ".pdf")

        else:
            if(buildingName in impossiblebuildings.identifier.values):
                reporter.notSimulated_file(title, building)
                eachPDF.append("PDF docs - Not Paged/" + title + ".pdf")

eP-EAZK-001
eP-EAZK-002
eP-EAZK-003
eP-EAZK-004
eP-EAZK-005
eP-EAZK-006
eP-EAZK-007
eP-EAZK-008
eP-EAZK-009
eP-EAZK-010
eP-EAZK-011
eP-EAZK-012
eP-EAZK-013
eP-EAZK-014
eP-EAZK-015
eP-EAZK-016
eP-EAZK-017
eP-EAZK-018
eP-EAZK-019
eP-EAZK-020
eP-EAZK-021
eP-EAZK-022
eP-EAZK-023
eP-EAZK-024
eP-EAZK-025
eP-EAZK-026
eP-EAZK-027
eP-EAZK-028
eP-EAZK-029
eP-EAZK-030
eP-EAZK-031
eP-EAZK-032
eP-EAZK-033
eP-EAZK-034
eP-EAZK-035
eP-EAZK-036
eP-EAZK-037
eP-EAZK-038
eP-EAZK-039
eP-EAZK-040
eP-EAZK-041
eP-EAZK-042
eP-EAZK-043
eP-EAZK-044
eP-EAZK-045
eP-EAZK-046
eP-EAZK-047
eP-EAZK-048
eP-EAZK-049
eP-EAZK-050
eP-EAZK-051
eP-EAZK-052
eP-EAZK-053
eP-EAZK-054
eP-EAZK-055
eP-EAZK-056
eP-EAZK-057
eP-EAZK-058
eP-EAZK-059
eP-EAZK-060
eP-EAZK-061
eP-EAZK-062
eP-EAZK-063
eP-EAZK-064
eP-EAZK-065
eP-EAZK-066
eP-EAZK-067
eP-EAZK-068
eP-EAZK-069
eP-EAZK-070
eP-EAZK-071
eP-EAZK-072
eP-EAZK-073
eP-EAZK-074
eP-EAZK-075
eP-EAZK-076
eP-EAZK-077
eP-EAZK-078
eP-EAZK-079
eP-EAZK-080
eP-EAZK-081
eP-EAZK-082
eP-EAZK-083
eP-E

# **TOC and Merge**

In [5]:
coverPath = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\_Final report\UC4 - Portada entregable.pdf"
pathContents = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\_Final report\Contents of this report - Watermarked.pdf"
pathSummary = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\_Final report\Buildings summary - Watermarked.pdf"
# basePathResults = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Samples and Test\Automated Report Test\PDF docs"
basePathResults = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Samples and Test\Automated Report Test\PDF docs - compressed"
outputPath =  r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\_Final report\Merged.pdf"

from ReportTemplates.CanvasWriter import CanvasWriter
CanvasWriter.mergeAndTOC(coverPath, pathContents, pathSummary, basePathResults, outputPath, reportTitle, logo)

# **Test bench**

In [20]:
image1 = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Samples and Test\Automated Report Test\MapResults\eP-EAZK-373_ZoomOut.png"
image2 = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Samples and Test\Automated Report Test\MapResults\eP-EAZK-373_ZoomIn.png"
image3 = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Samples and Test\Automated Report Test\MapResults\eP-EAZK-373_Planes.png"
image4 = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Samples and Test\Automated Report Test\MapResults\eP-EAZK-373_LiDAR.png"
image5 = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Samples and Test\Automated Report Test\MapResults\eP-EAZK-373_EnergyNumbered.png"
image6 = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Samples and Test\Automated Report Test\MapResults\eP-EAZK-373_EnergyDensity.png"

from PIL import Image

foo = Image.open(image3)  # My image is a 200x374 jpeg that is 102kb large
print(foo.size)  # (200, 374)

# downsize the image with an ANTIALIAS filter (gives the highest quality)
foo = foo.resize((250, 250))

image1_opt = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Samples and Test\Automated Report Test\MapResults\eP-EAZK-373_ZoomOut_optimized.png"
image2_opt = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Samples and Test\Automated Report Test\MapResults\eP-EAZK-373_ZoomIn_optimized.png"
image3_opt = r"C:\Users\jaasb\INVESTIGO\BEE Group\eplanet shared\Programa Final\Samples and Test\Automated Report Test\MapResults\eP-EAZK-373_Planes_optimized.png"

foo.save(image3, optimize=True, quality = 95)  # The saved downsized image size is 22.9kb

(600, 600)


In [24]:
import os # Interface with system directories
import pandas as pd
import numpy as np

pd.set_option('mode.chained_assignment', None) # For peace of mind and ease of debugging (gets rid of warning)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.collections import PatchCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable


from shapely import Polygon
from shapely import wkt
from shapely import affinity
import geopandas as gpd
import pyproj # Convert building and cadastre to new coordinate system

resultsPath = "C:/Users/jaasb/INVESTIGO/BEE Group/eplanet shared/Programa Final/_Final Results/"
mapResultsPath = "C:/Users/jaasb/INVESTIGO/BEE Group/eplanet shared/Programa Final/Samples and Test/Automated Report Test/MapResults/"

class MapsPlotter:
    def __init__(self, resultsPath, mapImagesPath):
        self.resultsPath = resultsPath
        self.mapImagesPath = mapImagesPath

    def plotEnergy(self, building, numbered=True, source=3035, target=5514):
        sunSimulationPath = self.resultsPath + building.identifier.values[0] + "/06 - PySAM Simulation/Summary.csv"
        if(os.path.isfile(sunSimulationPath)):
            # Load and prepare data
            plt.rcParams["figure.figsize"] = (6,6)
            centerx = building.x[0]
            centery = building.y[0]
            
            processedResultsPath = self.resultsPath + building.identifier.values[0] + "/04 - Plane Processing/01 - Results"
            planedf = pd.read_csv(processedResultsPath + "/PlaneList_" + building.identifier[0] + ".csv")

            segmented_path = self.resultsPath + building.identifier.values[0] + "/01 - Segmented Buildings"
            buildingPoints = pd.read_csv(segmented_path + "/" + building.identifier[0] + ".csv", header=None)
            buildingPoints = buildingPoints.rename(columns={0:"x", 1:"y", 2:"z"})
            
            minx = buildingPoints.x.min()
            miny = buildingPoints.y.min()
            buildingPoints.x = buildingPoints.x - minx
            buildingPoints.y = buildingPoints.y - miny
            
            pysamResultsPath = self.resultsPath + building.identifier.values[0] + "/06 - PySAM Simulation"
            pysam_df = pd.read_csv(pysamResultsPath + "\Summary.csv")

            # Plot function
            
            if(numbered):
                savename = self.mapImagesPath + building.identifier[0] + "_EnergyNumbered.png"
            else:
                savename = self.mapImagesPath + building.identifier[0] + "_EnergyDensity.png"
            cmap = mpl.cm.plasma #coolwarm, plasma, rainbow, cividis, YlOrRd

            bounds = list(range(100, 241, 20))
            norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
            sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm) 
            sm.set_array([]) 

            fig, ax = plt.subplots()

            pySAMOffsetX = (centerx - minx) + 8.70 
            pySAMOffsetY = (centery - miny) - 0.22

            for i in range(len(planedf)):
                currentdf = pysam_df[pysam_df.roofID==i]

                x = currentdf.x + pySAMOffsetX
                y = currentdf.y + pySAMOffsetY
                z = currentdf.ac_annual/currentdf.area
                
                if(len(x.unique()) > 1 and len(y.unique()) > 1):
                    cntr1 = ax.tricontourf(x, y, z, cmap=cmap, norm=norm, alpha=1)

                polygon = wkt.loads(planedf.trimmedPolygon[i])
                exterior_x, exterior_y = polygon.exterior.xy
                exterior_x = exterior_x - minx
                exterior_y = exterior_y - miny
                
                if(numbered):
                    plt.plot(exterior_x, exterior_y, "black", linewidth=2)
                else:
                    plt.plot(exterior_x, exterior_y, "black", linewidth=1)

                # Create a list of interior rings
                interior_x, interior_y = [], []
                for interior in polygon.interiors:
                    x, y = interior.xy
                    interior_x.append(x)
                    interior_y.append(y)

                for j in range(len(interior_x)):
                    hole_x = interior_x[j] - minx
                    hole_y = interior_y[j] - miny
                    plt.fill(hole_x, hole_y, c="white")
                    plt.plot(hole_x, hole_y, "black", linewidth=1)

                # Annotate
                if(numbered):
                    planeCenter = polygon.centroid
                    ax.annotate(str(i), (planeCenter.x - minx, planeCenter.y - miny), size=20, color=(0.1,0.1,0.1,1), backgroundcolor=(1,1,1,0.65), fontname="fantasy")

            # Apply mask to hide outside-cadastre areas
            cadastrePath = segmented_path + "/" + building.identifier[0]  + ".gpkg"
            cadastreLimits = gpd.read_file(cadastrePath)
            cadastreLimits = cadastreLimits.geometry[0]

            if(type(cadastreLimits) != type(Polygon())):
                cadastreLimits = cadastreLimits.geoms[0]

            source_crs = 'epsg:' + str(source) # Coordinate system of the buildings (default=?)
            target_crs = 'epsg:' + str(target) # Coordinate system of LiDAR data (default=Czech)
            global_to_czech = pyproj.Transformer.from_crs(source_crs, target_crs)

            cadastreLimitsCzech = global_to_czech.transform(cadastreLimits.exterior.coords.xy[1], cadastreLimits.exterior.coords.xy[0])

            p = []

            for j in range(len(cadastreLimitsCzech[0])):
                p.append((cadastreLimitsCzech[0][j], cadastreLimitsCzech[1][j]))

            actualLimitsCadastre = Polygon(p)
            actualLimitsCadastre = affinity.translate(actualLimitsCadastre, -minx, -miny)
            

            coords = ((min(actualLimitsCadastre.exterior.xy[0]) - 0.25, min(actualLimitsCadastre.exterior.xy[1]) - 0.25), 
                        (min(actualLimitsCadastre.exterior.xy[0]) - 0.25, max(actualLimitsCadastre.exterior.xy[1]) + 0.25), 
                        (max(actualLimitsCadastre.exterior.xy[0]) + 0.25, max(actualLimitsCadastre.exterior.xy[1]) + 0.25), 
                        (max(actualLimitsCadastre.exterior.xy[0]) + 0.25, min(actualLimitsCadastre.exterior.xy[1]) - 0.25), 
                        (min(actualLimitsCadastre.exterior.xy[0]) - 0.25, min(actualLimitsCadastre.exterior.xy[1]) - 0.25))
            rectangle = Polygon(coords)

            poly = rectangle.difference(actualLimitsCadastre)
            
            path = Path.make_compound_path(
                Path(np.asarray(poly.exterior.coords)[:, :2]),
                *[Path(np.asarray(ring.coords)[:, :2]) for ring in poly.interiors])

            patch = PathPatch(path, color='w')
            collection = PatchCollection([patch], color='w')
            
            ax.add_collection(collection, autolim=True)

            # fig.colorbar(sm,  ax=ax)
            ax.scatter(buildingPoints.x, buildingPoints.y, c="gray", alpha=0.05)
            # plt.subplots_adjust(hspace=0.5)
            ax.set_aspect('equal', adjustable='box')
            ax.set_xlim(min(ax.get_xlim()[0], ax.get_ylim()[0]), max(ax.get_xlim()[1], ax.get_ylim()[1]))
            ax.set_ylim(min(ax.get_xlim()[0], ax.get_ylim()[0]), max(ax.get_xlim()[1], ax.get_ylim()[1]))
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
            ax.set_xlabel('x (m)')  
            ax.set_ylabel('y (m)')  
            
            ax.grid(alpha=0.25)
            
            fig.savefig(savename)

            plt.close()

plotter = MapsPlotter(resultsPath, mapResultsPath)
plotter.plotEnergy(building, numbered=False)
plotter.plotEnergy(building, numbered=True)